## Import modules

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir("/home/ubuntu/lavender_floor_height/GA-floor-height")  # Set this to your repository folder root to allow loading other modules

In [ ]:
import geopandas as gpd
import math
from PIL import Image
import pandas as pd
import glob
from GSV import geometry

## Set input and output folders

In [ ]:
# input_buildings_pano_meta='/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata.geojson' # input buildings points with panorama metadata joined
# input_folder='/mnt/floorheightvolume/panorama_Wagga' # input folder containing panorama images
# out_folder='/mnt/floorheightvolume/panorama_Wagga_clipped' # output folder
# out_building_pano_meta='/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping.geojson'
# os.makedirs(out_folder, exist_ok=True)

In [ ]:
input_buildings_pano_meta='/mnt/floorheightvolume/all_buildings/Final_Wagga_pano_metadata.geojson' # input buildings points with panorama metadata joined
input_folder='/mnt/floorheightvolume/panorama_Wagga' # input folder containing panorama images
out_folder='/mnt/floorheightvolume/panorama_Wagga_clipped' # output folder
out_building_pano_meta='/mnt/floorheightvolume/all_buildings/Final_Wagga_pano_metadata_clipping.geojson'
os.makedirs(out_folder, exist_ok=True)

## Load metadata file

In [ ]:
# Read the JSON file
epsg_trajectory='EPSG:7844' # GDA2020
gdf_meta=gpd.read_file(input_buildings_pano_meta).to_crs(epsg_trajectory)
gdf_meta.head()

In [ ]:
gdf_meta=gdf_meta[gdf_meta["USAGE"]=="Residential"].reset_index(drop=True)
gdf_meta

In [ ]:
gdf_meta.Heading_deg

In [ ]:
# declination_deg=11.87
# declination_deg=0
gdf_meta['Heading_deg'] *= -1
# gdf_meta['Heading_deg']+= declination_deg
gdf_meta.loc[gdf_meta['Heading_deg'] < 0, 'Heading_deg'] += 360
# gdf_meta['Heading_deg']+= 180

In [ ]:
gdf_meta.columns

In [ ]:
gdf_meta.Heading_deg

### Test one example

In [ ]:
i=0

In [ ]:
pano_id=gdf_meta.at[i,'FRAMEID']
pano_id

In [ ]:
ufi=gdf_meta.at[i,'UFI']
ufi

## Read in pano
Allow large image to be loaded:

In [ ]:
Image.MAX_IMAGE_PIXELS=None

In [ ]:
pano_file=glob.glob(os.path.join(input_folder,'*'+pano_id+"*.jpg"))[0]
pano_img=Image.open(pano_file)
pano_file

## Localise building of interest

House location (horizontal pixel range) within pano image:

In [ ]:
location = geometry.localize_house_in_panorama(lat_c=gdf_meta.at[i,'LATITUDE'], 
                                    lon_c=gdf_meta.at[i,'LONGITUDE'], 
                                    lat_house=gdf_meta.geometry.iloc[i].y, 
                                    lon_house=gdf_meta.geometry.iloc[i].x,
                                    beta_yaw_deg=gdf_meta.at[i,'Heading_deg'], 
                                    Wim=pano_img.width,angle_extend=40)
location

Record locations:

In [ ]:
gdf_meta_updated=gdf_meta.copy()
if 'house_loc_left' not in gdf_meta_updated.columns:
    gdf_meta_updated['house_loc_left']=None
if 'house_loc_right' not in gdf_meta_updated.columns:
    gdf_meta_updated['house_loc_right']=None
gdf_meta_updated.at[i,'house_loc_left']=location['horizontal_pixel_range_house'][0]
gdf_meta_updated.at[i,'house_loc_right']=location['horizontal_pixel_range_house'][1]

## Clip panorama image to localised area
- Horizontally clipped to calculated pixel range
- Vertically clipped based on hard-coded upper and lower crop proportions range

In [ ]:
upper_crop=0.25
lower_crop=0.6

In [ ]:
pano_img_clipped=pano_img.crop(box=(location['horizontal_pixel_range_house'][0],
                   upper_crop*pano_img.height,
                   location['horizontal_pixel_range_house'][1],
                   lower_crop*pano_img.height
                   ))
pano_img_clipped

### Save clipped images
* added building UFI to avoid duplication of panoramas

In [ ]:
pano_img_clipped.save(os.path.join(out_folder,os.path.basename(pano_file).split('.')[0]+'_'+str(ufi)+'.jpg'), "jpeg")

## Batch clipping

In [ ]:
# pano_ids=gdf_meta['FRAMEID'][gdf_meta['FRAMEID'].notna()]
for i in range(len(gdf_meta)):
    pano_id=gdf_meta.at[i,'FRAMEID']
    print('frame id: ',pano_id)
    ufi=gdf_meta.at[i,'UFI']
    if not pd.isna(pano_id):
        try:
            pano_file=glob.glob(os.path.join(input_folder,'*'+pano_id+"*.jpg"))[0]
            out_clipped_pano=os.path.join(out_folder,os.path.basename(pano_file).split('.')[0]+'_'+str(ufi)+'.jpg')
            # read in pano image
            if os.path.exists(pano_file):
                pano_img=Image.open(pano_file)
                # localise house of interest
                location = geometry.localize_house_in_panorama(lat_c=gdf_meta.at[i,'LATITUDE'], 
                                            lon_c=gdf_meta.at[i,'LONGITUDE'], 
                                            lat_house=gdf_meta.geometry.iloc[i].y, 
                                            lon_house=gdf_meta.geometry.iloc[i].x,
                                            beta_yaw_deg=gdf_meta.at[i,'Heading_deg'], 
                                            Wim=pano_img.width,angle_extend=40)
                gdf_meta_updated.at[i,'house_loc_left']=location['horizontal_pixel_range_house'][0]
                gdf_meta_updated.at[i,'house_loc_right']=location['horizontal_pixel_range_house'][1]
                try:
                    # clip pano image
                    pano_img_clipped=pano_img.crop(box=(location['horizontal_pixel_range_house'][0],
                                upper_crop*pano_img.height,
                                location['horizontal_pixel_range_house'][1],
                                lower_crop*pano_img.height
                                ))
                    pano_img_clipped.save(out_clipped_pano)
                except Exception as e:
                    print(e)
            else:
                print('pano file missing')
        except Exception as e:
            print(e)
    else:
        print('no pano file associated with building points')

### Save updated building file with clipping range

In [ ]:
gdf_meta_updated.to_file(out_building_pano_meta)